In [150]:
using BenchmarkTools

In [223]:
module sp

import Base: *, +, ^, promote_rule, convert, show

immutable Variable{Name}
end

show(io::IO, v::Variable{Name}) where Name = print(io, Name)

immutable Monomial{N, V}
    exponents::NTuple{N, Int64}
end

format_exponent(e) = e == 1 ? "" : "^$e"

function show(io::IO, m::Monomial)
    for (i, v) in enumerate(variables(m))
        if m.exponents[i] != 0
            print(io, v, format_exponent(m.exponents[i]))
        end
    end
end

variables(::Type{Monomial{N, V}}) where {N, V} = V
variables(m::Monomial) = variables(typeof(m))

Monomial(v::Variable{Name}) where Name = Monomial{1, (Name,)}((1,))

*(v1::Variable{N1}, v2::Variable{N2}) where {N1, N2} = Monomial{2, (N1, N2)}((1, 1))
*(v1::Variable{N}, v2::Variable{N}) where N = Monomial{1, (N,)}((2,))

immutable Term{T, MonomialType <: Monomial}
    coefficient::T
    monomial::MonomialType
end

format_coefficient(c) = c == 1 ? "" : string(c)
show(io::IO, t::Term) = print(io, format_coefficient(t.coefficient), t.monomial)

Term(m::Monomial) = Term(1, m)
Term(v::Variable) = Term(Monomial(v))

@generated function convert(::Type{Term{T, Mono1}}, t::Term{T, Mono2}) where {T, Mono1, Mono2}
    args = Any[0 for v in variables(Mono1)]
    for (j, var) in enumerate(variables(Mono2))
        I = find(v -> v == var, variables(Mono1))
        if isempty(I)
            throw(InexactError())
        elseif length(I) > 1
            error("Duplicate variables in destination $Mono1")
        end
        args[I[1]] = :(t.monomial.exponents[$j])
    end
    quote
        Term{$T, $Mono1}(t.coefficient, 
            Monomial{$(length(args)), $(variables(Mono1))}($(Expr(:tuple, args...))))
    end
end
    
        
immutable Polynomial{M, TermType <: Term}
    terms::NTuple{M, TermType}
end


function show(io::IO, p::Polynomial)
    if !isempty(p.terms)
        print(io, p.terms[1])
        for i in 2:length(p.terms)
            print(io, " + ", p.terms[i])
        end
    end
end
    

@generated function promote_rule(::Type{Term{T, Mono1}}, ::Type{Term{T, Mono2}}) where {T, Mono1, Mono2}
    vars = Set{Symbol}()
    for v in variables(Mono1)
        push!(vars, v)
    end
    for v in variables(Mono2)
        push!(vars, v)
    end
    vars = Tuple(sort(collect(vars)))
    quote
        Term{T, Monomial{$(length(vars)), $(vars)}}
    end
end

(+)(t1::Term{T, Mono}, t2::Term{T, Mono}) where {T, Mono} = Polynomial((t1, t2))
(+)(t1::Term{T, Mono1}, t2::Term{T, Mono2}) where {T, Mono1, Mono2} = +(promote(t1, t2)...)
(+)(m1::Monomial, m2::Monomial) = Term(m1) + Term(m2)
(+)(v1::Variable, v2::Variable) = Term(v1) + Term(v2)
*(x::Number, m::Monomial) = Term(x, m)
*(x::Number, v::Variable) = x * Monomial(v)
^(v::Variable{Name}, x::Integer) where Name = Monomial{1, (Name,)}((x,))

end

IJulia.sp

In [224]:
x = sp.Variable{:x}()
y = sp.Variable{:y}()

y

In [228]:
@benchmark 3 * ($x * $y) + 2 * ($x ^2)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.264 ns (0.00% GC)
  median time:      2.268 ns (0.00% GC)
  mean time:        2.287 ns (0.00% GC)
  maximum time:     10.062 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [229]:
p = 3 * (x * y) + 2 * (x ^ 2)

3xy + 2x^2

In [214]:
sp.variables(p.terms[1].monomial)

(:x, :y)

In [205]:
p.terms[1].monomial.exponents

(1,)

In [199]:
x

x

In [200]:
@benchmark $x + $y

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.253 ns (0.00% GC)
  median time:      2.260 ns (0.00% GC)
  mean time:        2.281 ns (0.00% GC)
  maximum time:     14.022 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%